In [1]:
from tradealgorithm.tradealgorithm import TradeAlgorithm
from tradealgorithm.tradealgorithmfactory import TradeAlgorithmFactory
from pricer.pricer_factory import PricerFactory
from pricer.pricer import Pricer
from ranker.ranker_factory import RankerFactory
from ranker.ranker import Ranker
from classifier.classifier_factory import ClassifierFactory
from classifier.classifier import Classifier
from datetime import datetime
from tqdm import tqdm
from alpaca_api.alpaca_api import AlpacaApi
from time import sleep
import pandas as pd

In [2]:
trade_algo = TradeAlgorithmFactory.build(TradeAlgorithm.RRR_BETA)
alp = AlpacaApi()

In [3]:
ranker_class = Ranker.NONE
classifier_class = Classifier.NONE
current = False

In [7]:
start = datetime(2001,1,1)
end = datetime(2023,1,1)
current_date = datetime.now()
# week = current_date.week
week = 30
positions = 20

In [8]:
pricer_classes = [] 
# pricer_classes.append(Pricer.DAILY_STOCK_ROLLING)
# pricer_classes.append(Pricer.WEEKLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.MONTHLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.QUARTERLY_STOCK_ROLLING)
# pricer_classes.append(Pricer.DAILY_STOCK_WINDOW)
pricer_classes.append(Pricer.WEEKLY_STOCK_WINDOW)
# pricer_classes.append(Pricer.MONTHLY_STOCK_WINDOW)
# pricer_classes.append(Pricer.QUARTERLY_STOCK_WINDOW)

In [9]:
for pricer_class in tqdm(pricer_classes):
    try:
        trade_algo.initialize(pricer_class,ranker_class,classifier_class,start,end,current_date)
        trade_algo.initialize_classes()
        final = trade_algo.pull_recommendations()
        if final.index.size > 0:
            final = final[final["week"]==week].sort_values("weekly_delta",ascending=False).head(positions)
#             trade_algo.db.cloud_connect()
#             trade_algo.db.store("proposals",final)
#             trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("proposals",final)
            trade_algo.db.disconnect()
#             account = alp.live_get_account()
#             cash = float(account.cash)
            cash = 100
            # executing order
            order_data = []
            for row in final.iterrows():
                try:
                    ticker = "BTC/USD" if row[1]["ticker"] == "BTC" else row[1]["ticker"] 
                    amount = round(cash / (positions))
                    print(ticker,amount)
#                     order_data.append(alp.live_market_order(ticker,amount))
#                     sleep(1)
                except Exception as e:
                    trade_algo.db.connect()
                    trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"error":str(e)}]))
                    trade_algo.db.disconnect()
            order_dicts = pd.DataFrame([json.loads(order_d.json()) for order_d in order_data])
#             trade_algo.db.cloud_connect()
#             trade_algo.db.store("orders",order_dicts)
#             trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("orders",order_dicts)
            trade_algo.db.disconnect()
#         trade_algo.db.cloud_connect()
#         trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"complete"}]))
#         trade_algo.db.disconnect()
        trade_algo.db.connect()
        trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"complete"}]))
        trade_algo.db.disconnect()
#         sleep(300)
    except Exception as e:
#             trade_algo.db.cloud_connect()
#             trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"incomplete"}]))
#             trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"status":"trade","error":str(e)}]))
#             trade_algo.db.disconnect()
            trade_algo.db.connect()
            trade_algo.db.store("iterations",pd.DataFrame([{"date":str(datetime.now()),"status":"incomplete"}]))
            trade_algo.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"status":"trade","error":str(e)}]))
            trade_algo.db.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 25.62it/s]

AAP 5
VNO 5
CCL 5
LUMN 5
TSLA 5
ZION 5
NVDA 5
NCLH 5
CTLT 5
CMA 5
DG 5
ADBE 5
MTCH 5
ODFL 5
LNC 5
CZR 5
AVGO 5
BXP 5
DAL 5
GNRC 5
wsw_n_n_b_r orders documents must be a non-empty list


In [10]:
final

,year,week,ticker,weekly_delta,weekly_delta_sign,position,iteration
20,2023,30,AAP,0.642578,1.0,0,141.0
0,2023,30,VNO,0.407061,1.0,0,359.0
1,2023,30,CCL,0.388895,1.0,1,359.0
21,2023,30,LUMN,0.383425,1.0,1,141.0
2,2023,30,TSLA,0.346503,1.0,2,359.0
3,2023,30,ZION,0.336654,1.0,3,359.0
4,2023,30,NVDA,0.334304,1.0,4,359.0
5,2023,30,NCLH,0.323832,1.0,5,359.0
6,2023,30,CTLT,0.319100,1.0,6,359.0
7,2023,30,CMA,0.313896,1.0,7,359.0
